In [ ]:
set.seed(2019)

# PERFORMING FORWARD SELECTION TO FIND BEST PREDICTOR VARIABLES

potential_predictors <- colnames(upsampled_tennis_train |>
                                     select(-round))

cv_accuracies <- tibble(num_predictors = integer(),
                        model_string = character(),
                        accuracy = numeric())

tennis_fs_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = tune()) |>
    set_engine("kknn") |>
    set_mode("classification")

tennis_vfold <- vfold_cv(upsampled_tennis_train, v = 5, strata = round)

n_potential_preds <- length(potential_predictors)

selected_preds <- c()

for (i in 1:n_potential_preds) {
    accs <- list()
    models <- list()
    for (j in 1:length(potential_predictors)) {
        preds_new <- c(selected_preds, potential_predictors[[j]])
        model_string <- paste("round", "~", paste(preds_new, collapse = "+"))

        tennis_recipe <- recipe(as.formula(model_string), 
                                data = upsampled_tennis_train) |>
                          step_scale(all_predictors()) |>
                          step_center(all_predictors())

        acc <- workflow() |>
          add_recipe(tennis_recipe) |>
          add_model(tennis_fs_spec) |>
          tune_grid(resamples = tennis_vfold, grid = 10) |>
          collect_metrics() |>
          filter(.metric == "accuracy") |>
          summarize(mx = max(mean))
        acc <- acc$mx |> unlist()

        accs[[j]] <- acc
        models[[j]] <- model_string
    }
    jstar <- which.max(unlist(accs))
    cv_accuracies <- cv_accuracies |> 
      add_row(num_predictors = i, 
              model_string = models[[jstar]], 
              accuracy = accs[[jstar]])
    selected_preds <- c(selected_preds, potential_predictors[[jstar]])
    potential_predictors <- potential_predictors[-jstar]
}
cv_accuracies

# shows no significant increase in accuracy for any predictors other than w_over_l_point_ratio

In [1]:
write_csv(cv_accuracies, "forward_selection.csv")

ERROR: Error in write_csv(cv_accuracies, "forward_selection.csv"): could not find function "write_csv"
